In [1]:
import tensorflow as tf
#import tensorflow.contrib.keras as keras
from pathlib import Path
from tensorflow.keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

import os
import matplotlib.pyplot as plt
import random
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt


print("Done with library declaration, Current version of Tensorflow is: ", tf.__version__)

Done with library declaration, Current version of Tensorflow is:  2.16.1


In [ ]:
class_names = ["aluminem_cans", "cardboard", "food", "generic_trash", "glass_bottle", "hazardous_waste", "medical_waste", "paper", "plastic_bottle", "plastic_not_recylable", "plastic_recylable"]

image_root_path = Path("data_organized")
train_root_path = image_root_path / "train"
test_root_path = image_root_path / "test"


os.makedirs(train_root_path, exist_ok=True)
os.makedirs(test_root_path, exist_ok=True)

train_count = 0
test_count = 0
for class_name in class_names:
    imagepath = image_root_path / class_name
    train_graypath = train_root_path / class_name
    test_graypath = test_root_path / class_name
  
    os.makedirs(train_graypath, exist_ok=True)
    os.makedirs(test_graypath, exist_ok=True)
 
    file_list = os.listdir(imagepath)
    num_files_train = int(len(file_list) * 0.8)
    num_files_test = len(file_list) - num_files_train
    test_files = random.sample(file_list, num_files_test)

    for file in file_list:
        img_path = imagepath / file
        if file in test_files:
            gray_path = test_graypath / file
            test_count += 1
        else:
            gray_path = train_graypath / file
            train_count += 1
        im = Image.open(img_path)
        img = im.resize((32, 32))
        gray = img.convert('L')
        gray.save(gray_path, "JPEG")
        
print(test_count)
print(train_count)

In [ ]:
model = Sequential([
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(32,32, 1)),
    MaxPooling2D(pool_size=2),

    Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),

    Flatten(),

    Dense(11, activation='softmax')
])



In [ ]:
batch_size = 45
epochs = 60
model.compile(optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

model.summary()

In [ ]:
train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)


train_generator = train_gen.flow_from_directory(
    train_root_path,
    target_size=(32, 32),
    color_mode='grayscale',
    class_mode='categorical'
)

test_generator = test_gen.flow_from_directory(
    test_root_path,
    target_size=(32, 32),
    color_mode='grayscale',
    class_mode='categorical'
)


batch_size = 32
epochs = 10
steps_per_epoch = train_count // batch_size
validation_steps = test_count // batch_size





early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=7)


history = model.fit(
    train_generator,
    validation_data=test_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    # validation_steps=validation_steps
    # callbacks=[early_stopping]
)


test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy: {} Test Loss: {} '.format(test_acc*100, test_loss))







In [ ]:
train_acc = history.history['accuracy'] 
val_acc = history.history['val_accuracy'] 
train_loss = history.history['loss'] 
val_loss = history.history['val_loss'] 

epochs = range(1, len(train_acc) + 1)

plt.plot(epochs, train_acc, 'g*-', label = 'Training accuracy')
plt.plot(epochs, val_acc, 'r', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, train_loss, 'g*-', label = 'Training loss')
plt.plot(epochs, val_loss, 'r', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()